In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import time
from snowflake.snowpark.session import Session
import configparser

import warnings
warnings.filterwarnings("ignore")

config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [2]:
sales_bangalore_2022 = session.table("SOL_ASSORTMENT_PLANNING_COMB").to_pandas()

In [3]:
sales_bangalore_2022.head()

,TRANSACTION_DATE,MNTH_CODE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,DISTRIBUTOR_CODE,PRODUCT_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,CATEGORY,VARIANT,BRAND
0,2023-07-01,202203,460.91,3,0.0027,460.909091,DB0706,PRD0041,OL81278,Montclair,New Jersey,Dolphin,Str1,Soap,Beauty Soap,Charcoal
1,2023-07-01,202203,277.14,32,8.32E-04,285.714286,DB0706,PRD0069,OL206924,Breckenridge,Colorado,Dolphin,Str2,Perfume and Deodrants,Female Deodrant,Arctic blue
2,2023-07-01,202203,636.36,4,0.0012,636.363636,DB0706,PRD0078,OL81756,Tucson,Arizona,City Center,Str1,Lotion,Head Lotion,Saffron
3,2023-07-01,202203,138.57,16,4.00E-04,142.857143,DB0209,PRD0147,OL238948,San Pedro,California,Orange,Str5,Kids Care,Baby Cream,Mint
4,2023-07-01,202203,142.86,16,4.16E-04,142.857143,DB0110,PRD0069,OL81622,Orange,Connecticut,Silver,Str5,Perfume and Deodrants,Female Deodrant,Arctic blue


In [4]:
train_final = session.table("SOL_ASSORTMENT_PLANNING_TRAIN_DATA_MODEL").to_pandas()
train_final.head(1)

,S_NO,UNIQUE_ID,FREQUENCYM,SALES_VALUE_AVG,SALES_UNITS_AVG,SALES_INDICATOR,SCHEME_AMOUNT_PERPRODUCT
0,0,PRD0002-OL10330,0.6,0.004323138,0.00040404,1,0.000465


In [5]:
master_data = sales_bangalore_2022.copy()

In [6]:
master_data = sales_bangalore_2022.copy()
master_data['mnth_code'] = pd.to_numeric(master_data['MNTH_CODE'],errors ='coerce').astype(int)
df1=master_data.copy
master_data['unique_id']=master_data['PRODUCT_CODE']+'-'+ master_data['OUTLET_CODE']

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_final[['FREQUENCYM','SALES_VALUE_AVG','SCHEME_AMOUNT_PERPRODUCT']],train_final['SALES_INDICATOR'],test_size=0.3,random_state=42)